# LUNG NODULE IDENTIFICATION

### Import libraries

In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import rawpy
import os
import numpy as np
import skimage.io as io
import SimpleITK as sltk
import matplotlib.animation as animation
import imageio
import cv2
from IPython.display import Image
import warnings
import multiprocessing
import glob
import random
warnings.filterwarnings('ignore')
#%matplotlib inline

### Reading Data Files

#### Nodule Annotation File

In [30]:
path='/export/data/phddata/thaque/'
#path='D:/Data Science Bowl 2017/LUNA Dataset/'

In [31]:
df1=pd.read_csv(path+'CSVFILES/annotations.csv')
df1.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


#### False Positive Reduction File

In [32]:
df=pd.read_csv(path+'CSVFILES/candidates_V2.csv')
df.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


In [33]:
#Dataset is highly imbalanced
df['class'].value_counts()

0    753418
1      1557
Name: class, dtype: int64

#### Sample submission file

In [34]:
df2=pd.read_csv(path+'CSVFILES/sampleSubmission.csv')
df2.head()

,seriesuid,coordX,coordY,coordZ,probability
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.6,-175.3,-298.3,1.0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.7,-211.9,-227.1,0.8
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.6,-140.9,876.3,0.2
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.0,192.1,-391.0,0.5
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.4,172.4,-405.4,1.0


### Pre-processing

#### Resampling image to isometric form

A scan may have a pixel spacing of [2.5, 0.5, 0.5], which means that the distance between slices is 2.5 millimeters. For a different scan this may be [1.5, 0.725, 0.725], this can be problematic for automatic analysis (e.g. using ConvNets)! A common method of dealing with this is resampling the full dataset to a certain isotropic resolution. If we choose to resample everything to 1mm1mm1mm pixels we can use 3D convnets without worrying about learning zoom/slice thickness invariance. 
<br> -  [1] https://www.kaggle.com/akh64bit/full-preprocessing-tutorial; <br> -  [2] https://www.raddq.com/dicom-processing-segmentation-visualization-in-python/

The chest scans are produced by a variety of CT scanners, this causes a difference in spacing between voxels of the original scan. We rescaled and interpolated all CT scans so that each voxel represents a 1x1x1 mm cube. <br> -  [3] https://eliasvansteenkiste.github.io/machine%20learning/lung-cancer-pred/

In [36]:
def resample_image(itk_image, out_spacing=(1.0, 1.0, 1.0), is_label=False):
    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [int(np.round(original_size[0]*(original_spacing[0]/out_spacing[0]))),
                int(np.round(original_size[1]*(original_spacing[1]/out_spacing[1]))),
                int(np.round(original_size[2]*(original_spacing[2]/out_spacing[2])))]

    resample = sltk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sltk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sltk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sltk.sitkBSpline)

    return resample.Execute(itk_image)

### Generate Binary Mask and Crop Region of Interest

In [40]:
#Create a list of all CT scans in a folder
L=[]
for i in range(0,10):
    M=[]
    M=glob.glob(path+'subset'+str(i)+'/subset'+str(i)+'/*.mhd')
    L=L+M
len(L)

888

In [41]:
train_data=L[0:830]
len(train_data)

830

#### Remove scans that have too many nodules as the tensor runs out of memory

In [55]:
new_train_data=[]
number_nodules=[]
for i in range(0,len(train_data)):
    a=train_data[i].split('/')[len(train_data[i].split('/'))-1][:-4]
    sub_loop=len(df1[df1['seriesuid']==a])
    
    if (sub_loop<7):
        number_nodules.append(sub_loop)
        new_train_data.append(train_data[i])

In [56]:
len(new_train_data)

820

#### Function to convert images to arrays, generate related mask, and split scans into smaller slices

In [57]:
def load_images(file_list,start,end,size_split,min_slices_per_batch):
    
    '''
    file_list represent the list of the location of files on the drive
    start - represents the start index of picking the files from the file list
    end - represents the end index of picking the files from the file list
    split_size - represents the size of slices to be carved out of image
    min_slides_per_scan - represents the minimum slices of the image to be included in training; first slice with lung nodules are included and the remaining slices are filled with empty slices
    '''
    
    cropped_mask=[]
    cropped_image=[]
    
    for i in range(start,end):
        a=file_list[i].split('/')[len(file_list[i].split('/'))-1][:-4]
        
        nda0=sltk.ReadImage(file_list[i])
        nda1=resample_image(nda0)
        nda=sltk.GetArrayFromImage(nda1)
        
        #nda=(nda-nda.min())/(nda.max()-nda.min())*255
    
        AA = np.zeros((nda.shape[0],nda.shape[1], nda.shape[2]),dtype=int)
    
        X_org=np.array(nda1.GetOrigin())[0]
        Y_org=np.array(nda1.GetOrigin())[1]
        Z_org=np.array(nda1.GetOrigin())[2]
    
        sub_loop=len(df1[df1['seriesuid']==a])
                
        for w in range(0,sub_loop):
            radius = int(df1[df1['seriesuid']==a]['diameter_mm'].iloc[w]/2)
            x0=df1[df1['seriesuid']==a]['coordX'].iloc[w]
            y0=df1[df1['seriesuid']==a]['coordY'].iloc[w]
            z0=df1[df1['seriesuid']==a]['coordZ'].iloc[w]

            v_x0=int(x0-X_org)
            v_y0=int(y0-Y_org)
            v_z0=int(z0-Z_org)
            
            for x in range(v_z0-radius-1, v_z0+radius+1):
                for y in range(v_y0-radius-1, v_y0+radius+1):
                    for z in range(v_x0-radius-1, v_x0+radius+1):
                        deb = radius - abs(v_z0-x) - abs(v_y0-y) - abs(v_x0-z) 
                        if (deb)>=0:
                            AA[x,y,z] = int(1)
            
        zzz=(size_split//2+1,size_split//2+1)
        yyy=(size_split//2+1,size_split//2+1)
        xxx=(size_split//2+1,size_split//2+1)
        npad = (zzz,yyy,xxx)
            
        padded_mask = np.pad(AA, pad_width=npad, mode='constant', constant_values=int(0))
        padded_array =np.pad(nda, pad_width=npad,mode='constant',constant_values=int(0))
        
        count_captured_image=0
        #Crop region of interest
        for w in range(0,sub_loop):
            radius = int(df1[df1['seriesuid']==a]['diameter_mm'].iloc[w]/2)
            x0=df1[df1['seriesuid']==a]['coordX'].iloc[w]
            y0=df1[df1['seriesuid']==a]['coordY'].iloc[w]
            z0=df1[df1['seriesuid']==a]['coordZ'].iloc[w]

            v_x0=int(x0-X_org)+size_split//2+1
            v_y0=int(y0-Y_org)+size_split//2+1
            v_z0=int(z0-Z_org)+size_split//2+1
            
            #I don't know why i did this but it is fixing an error in loading files
            if  not ((abs(v_z0)<32 and v_z0<0)  or (abs(v_y0)<32 and v_y0<0) or (abs(v_x0)<32 and v_x0<0)):
                count_captured_image=count_captured_image+1
                cropped_mask.append(padded_mask[v_z0-32:v_z0+32,v_y0-32:v_y0+32,v_x0-32:v_x0+32])
                cropped_image.append(padded_array[v_z0-32:v_z0+32,v_y0-32:v_y0+32,v_x0-32:v_x0+32])
                                                  
        z_count=nda.shape[0]//size_split
        y_count=nda.shape[1]//size_split
        x_count=nda.shape[2]//size_split
                        
        #Capture some random empty slices from a scan
        if count_captured_image<min_slices_per_batch:
            select_count=(min_slices_per_batch-count_captured_image)//8+1
            z_selected=random.sample(list(range(0,z_count)),select_count)
            y_selected=random.sample(list(range(0,y_count)),select_count)
            x_selected=random.sample(list(range(0,x_count)),select_count)
            
            for ip in range(0,z_count):
                for jp in range(0,y_count):
                    for kp in range(0,x_count):
                        sum_pix=padded_mask[ip*size_split:(ip+1)*size_split,jp*size_split:(jp+1)*size_split,kp*size_split:(kp+1)*size_split].sum()
                        if (ip in z_selected and jp in y_selected and kp in x_selected):
                            cropped_mask.append(AA[ip*size_split:(ip+1)*size_split,jp*size_split:(jp+1)*size_split,kp*size_split:(kp+1)*size_split])
                            cropped_image.append(nda[ip*size_split:(ip+1)*size_split,jp*size_split:(jp+1)*size_split,kp*size_split:(kp+1)*size_split])    
                
        AA=None
        padded_array=None
        nda=None
        nda1=None
        padded_mask=None
        empty_mask=None
        empty_array=None
        nda0=None
    
    return cropped_image,cropped_mask

#### Function to batch generate images and related mask for training

In [58]:
def image_generator(files_path, batch_size,size_split,min_slices_per_batch):

    LUF = len(files_path)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < LUF:
            limit = min(batch_end, LUF)
            X,Y = load_images(files_path,batch_start,limit,size_split,min_slices_per_batch)
            X=np.asarray(X)
            Y=np.asarray(Y)
            data_X=X.reshape(X.shape[0],X.shape[1],X.shape[2],X.shape[3],1)
            data_Y=Y.reshape(Y.shape[0],Y.shape[1],Y.shape[2],Y.shape[3],1)
            
            yield (data_X,data_Y) #a tuple with two numpy arrays with batch_size samples     

            batch_start += batch_size   
            batch_end += batch_size

#### Function to generate validation dataset

In [59]:
def valid_data(file_list,start,end,size_split,min_slices_per_batch):
    
    cropped_image,cropped_mask=load_images(file_list,start,end,size_split,min_slices_per_batch)
        
    cropped_image=np.asarray(cropped_image)
    data_X=cropped_image.reshape(cropped_image.shape[0],cropped_image.shape[1],cropped_image.shape[2],cropped_image.shape[3],1)

    cropped_mask=np.asarray(cropped_mask)
    data_Y=cropped_mask.reshape(cropped_mask.shape[0],cropped_mask.shape[1],cropped_mask.shape[2],cropped_mask.shape[3],1)
    
    return data_X,data_Y

#### Generate validation dataset

In [60]:
slice_size=64
number_of_slices_per_scan=12
X,Y=valid_data(file_list=L,start=830,end=850,size_split=slice_size,min_slices_per_batch=number_of_slices_per_scan)

In [61]:
X.shape

(80, 64, 64, 64, 1)

In [62]:
Y.shape

(80, 64, 64, 64, 1)

In [ ]:
print('Starting U-NET training...')

## Image Segmentation

In [25]:
import numpy as np
import pandas as pd
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import warnings

from skimage import measure, morphology
from skimage.transform import resize

from keras.models import Model
from keras.layers import Input, Dense, Conv3D, MaxPooling3D, UpSampling3D, merge
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import SpatialDropout3D
from keras.layers import BatchNormalization
from keras.models import load_model

Using TensorFlow backend.


In [26]:
#Numbers below gets divided everytime we apply maxpooling; this can create issue when concatenating two models
NUM_SLIDES=64
IMG_HEIGHT=64
IMG_WIDTH=64
IMG_CHANNELS=1

In [27]:
def dice_coef(y_true, y_pred):
    smooth = .000000001
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.abs(K.sum(y_true_f * y_pred_f))
    return (2. * intersection) / (K.abs(K.sum(y_true_f)) + K.abs(K.sum(y_pred_f)) + smooth)
#Remove smooth from numerator

In [28]:
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [29]:
def recall_metric(y_true, y_pred):
    smooth = .000000001
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.abs(K.sum(y_true_f * y_pred_f))
    recall = (intersection) / (K.abs(K.sum(y_true_f)) + smooth)
    return recall

In [ ]:
# Build U-Net model
inputs = keras.layers.Input((NUM_SLIDES,IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
#s = keras.layers.Lambda(lambda x: x / 3095)(inputs)
 
c1 = keras.layers.Conv3D(16, kernel_size=(3,3,3), activation='relu',padding='same')(inputs)
c1 = keras.layers.SpatialDropout3D(0.3)(c1)
c1 = keras.layers.Conv3D(16, (3,3,3), activation='relu',padding='same')(c1)
#GlobalAveragePooling3D -- try this as well
p1 = keras.layers.MaxPooling3D((2,2,2))(c1)
p1 = BatchNormalization()(p1)
 
c2 = keras.layers.Conv3D(32, (3,3,3), activation='relu',padding='same')(p1)
c2 = keras.layers.SpatialDropout3D(0.3)(c2)
c2 = keras.layers.Conv3D(32, (3,3,3), activation='relu',padding='same')(c2)
p2 = keras.layers.MaxPooling3D((2,2,2))(c2)
p2 = BatchNormalization()(p2)

c3 = keras.layers.Conv3D(64, (3,3,3), activation='relu',padding='same')(p2)
c3 = keras.layers.SpatialDropout3D(0.3)(c3)
c3 = keras.layers.Conv3D(64, (3,3,3), activation='relu',padding='same')(c3)
p3 = keras.layers.MaxPooling3D((2, 2,2))(c3)
p3 = BatchNormalization()(p3)

c4 = keras.layers.Conv3D(128, (3,3,3), activation='relu',padding='same')(p3)
c4 = keras.layers.SpatialDropout3D(0.3)(c4)
c4 = keras.layers.Conv3D(128, (3,3,3), activation='relu', padding='same')(c4)
p4 = keras.layers.MaxPooling3D(pool_size=(2,2,2))(c4)
p4 = BatchNormalization()(p4)

c5 = keras.layers.Conv3D(256, (3,3,3), activation='relu',padding='same')(p4)
c5 = keras.layers.SpatialDropout3D(0.3)(c5)
c5 = keras.layers.Conv3D(256, (3,3,3), activation='relu', padding='same')(c5)
p5 = keras.layers.MaxPooling3D(pool_size=(2,2,2))(c5)
p5 = BatchNormalization()(p5)


c55 = keras.layers.Conv3D(512, (3,3,3), activation='relu',padding='same')(p5)
c55 = keras.layers.SpatialDropout3D(0.3)(c55)
c55 = keras.layers.Conv3D(512, (3,3,3), activation='relu',padding='same')(c55)
c55 = BatchNormalization()(c55)


u66 = keras.layers.Conv3DTranspose(256, (2,2,2), strides=(2,2,2), padding='same')(c55)
u66 = keras.layers.concatenate([u66, c5])
c66 = keras.layers.Conv3D(256, (3,3,3), activation='relu',padding='same')(u66)
c66 = keras.layers.SpatialDropout3D(0.3)(c66)
c66 = keras.layers.Conv3D(256, (3,3,3), activation='relu',padding='same')(c66)
c66 = BatchNormalization()(c66)

u6 = keras.layers.Conv3DTranspose(128, (2,2,2), strides=(2,2,2), padding='same')(c66)
u6 = keras.layers.concatenate([u6, c4])
c6 = keras.layers.Conv3D(128, (3,3,3), activation='relu',padding='same')(u6)
c6 = keras.layers.SpatialDropout3D(0.3)(c6)
c6 = keras.layers.Conv3D(128, (3,3,3), activation='relu',padding='same')(c6)
c6 = BatchNormalization()(c6)

u7 = keras.layers.Conv3DTranspose(64, (2, 2,2), strides=(2, 2,2), padding='same')(c6)
u7 = keras.layers.concatenate([u7, c3])
c7 = keras.layers.Conv3D(64, (3, 3,3), activation='relu',padding='same')(u7)
c7 = keras.layers.SpatialDropout3D(0.3)(c7)
c7 = keras.layers.Conv3D(64, (3, 3,3), activation='relu',padding='same')(c7)
c7 = BatchNormalization()(c7)

u8 = keras.layers.Conv3DTranspose(32, (2, 2,2), strides=(2, 2,2), padding='same')(c7)
u8 = keras.layers.concatenate([u8, c2])
c8 = keras.layers.Conv3D(32, (3, 3,3), activation='relu',padding='same')(u8)
c8 = keras.layers.SpatialDropout3D(0.3)(c8)
c8 = keras.layers.Conv3D(32, (3, 3,3), activation='relu',padding='same')(c8)
c8 = BatchNormalization()(c8)

u9 = keras.layers.Conv3DTranspose(16, (2, 2,2), strides=(2, 2,2), padding='same')(c8)
u9 = keras.layers.concatenate([u9, c1], axis=4)
c9 = keras.layers.Conv3D(16, (3, 3,3), activation='relu',padding='same')(u9)
c9 = keras.layers.SpatialDropout3D(0.3)(c9)
c9 = keras.layers.Conv3D(16, (3, 3,3), activation='relu',padding='same')(c9)
c9 = BatchNormalization()(c9)
 
outputs = keras.layers.Conv3D(1, (1, 1,1), activation='sigmoid')(c9)
 
model = keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss=dice_coef_loss, metrics=[dice_coef,recall_metric])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 64, 1 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 64, 64, 64, 1 448         input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout3d_1 (SpatialDro (None, 64, 64, 64, 1 0           conv3d_1[0][0]                   
__________________________________________________________________________________________________
conv3d_2 (Conv3D)               (None, 64, 64, 64, 1 6928        spatial_dropout3d_1[0][0]        
__________________________________________________________________________________________________
max_poolin

In [ ]:
model = load_model('temp_model.h5', custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef,'recall_metric':recall_metric})

In [ ]:
batch_size=1
checkpoint = keras.callbacks.ModelCheckpoint('temp_model.h5', save_weights_only=False)
model.fit_generator(image_generator(files_path=new_train_data,batch_size=batch_size,size_split=slice_size,min_slices_per_batch=number_of_slices_per_scan),steps_per_epoch=len(new_train_data)//batch_size,epochs=100,validation_data=(X,Y),verbose=1,callbacks=[checkpoint])

Epoch 1/10
551/820 [===================>..........] - ETA: 1:15:50 - loss: -0.0015 - dice_coef: 0.0015 - recall_metric: 0.7811

In [ ]:
model.save('my_model_refined_dice_10_100.h5')